<a href="https://colab.research.google.com/github/CocoRyse/DicomToCsv/blob/main/%D0%9C%D0%9A%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Загрузка библиотек в ноутбук 
!pip install pydicom

In [ ]:
# Стандартные бибилиотеки
import os
import pydicom # Библиотека для чтения DICOM файлов
from collections import OrderedDict
import csv

# Визуализация (понадобится позже, пока я не разобрался как png в csv хранить)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
print('Пример всех данных из заданного DICOM файла:\n')
# тут просто путь к DICOM файлу 
example_dicom = pydicom.dcmread('/content/CR000000.dcm')
print(example_dicom)

Пример всех данных из заданного DICOM файла:

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 200
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Computed Radiography Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.276.0.7230010.3.1.4.186081398.5076.1564393902.11925
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.276.0.7230010.3.0.3.6.4
(0002, 0013) Implementation Version Name         SH: 'OFFIS_DCMTK_364'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 144'
(0008, 0008) Image Type                          CS: ['DERIVED', 'PRIMARY']
(0008, 0013) Instance Creation Time              TM: ''
(0008, 0016) SOP Class UID                       UI: Computed Radiography Image Storage
(0008, 0018) SOP I

In [ ]:
def get_tags_in_files(dicom_file_path, tag_file_path):
    """
    Для файла с .dcm расширением находит все теги, а затем сохраняет их в качестве csv файла
    get_tags_in_files iterates over a directory, finds dicom files with
    a .dcm extension, and finds all unique dicom tag instances. it then
    writes the tags out as a csv file.
    Args:
        dicom_path (str): Путь до DICOM
        tag_file_path (str): Путь до .csv с результатом
    Returns:
        dict: Словарь с выгруженными тегами
    """
    # Создаём директорию для вывода
    if not os.path.exists(os.path.dirname(tag_file_path)):
        os.makedirs(os.path.dirname(tag_file_path))

    # Получаем теги
    tags_in_files = {}
    dicom_file = pydicom.read_file(dicom_file_path)
    for item in dicom_file:

        if item.keyword not in tags_in_files:
            group = "0x%04x" % item.tag.group
            element = "0x%04x" % item.tag.element
            tags_in_files[item.keyword] = group, element, item.keyword, item.name

    # Сортируем теги
    tags_in_files = OrderedDict(
        sorted(tags_in_files.items(), key=(lambda k: (k[1][0], k[1][1])))
    )

    # Записываем теги в .csv файл
    with open(tag_file_path, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["group", "element", "keyword", "name"])
        for item in tags_in_files:
            writer.writerow(tags_in_files[item])

    return tags_in_files

In [ ]:
get_tags_in_files('/content/CR000000.dcm', '/content/result.csv')

OrderedDict([('SpecificCharacterSet',
              ('0x0008',
               '0x0005',
               'SpecificCharacterSet',
               'Specific Character Set')),
             ('ImageType', ('0x0008', '0x0008', 'ImageType', 'Image Type')),
             ('InstanceCreationTime',
              ('0x0008',
               '0x0013',
               'InstanceCreationTime',
               'Instance Creation Time')),
             ('SOPClassUID',
              ('0x0008', '0x0016', 'SOPClassUID', 'SOP Class UID')),
             ('SOPInstanceUID',
              ('0x0008', '0x0018', 'SOPInstanceUID', 'SOP Instance UID')),
             ('StudyTime', ('0x0008', '0x0030', 'StudyTime', 'Study Time')),
             ('ContentTime',
              ('0x0008', '0x0033', 'ContentTime', 'Content Time')),
             ('AccessionNumber',
              ('0x0008', '0x0050', 'AccessionNumber', 'Accession Number')),
             ('RetrieveAETitle',
              ('0x0008', '0x0054', 'RetrieveAETitle', 'Retriev